In [40]:
import sqlite3
import pandas as pd

# sqlite3 module

In [4]:
# connection object
conn = sqlite3.connect('example.db')
type(conn)

sqlite3.Connection

In [6]:
[i for i in dir(conn) if i[0] != '_']

['DataError',
 'DatabaseError',
 'Error',
 'IntegrityError',
 'InterfaceError',
 'InternalError',
 'NotSupportedError',
 'OperationalError',
 'ProgrammingError',
 'Warning',
 'backup',
 'close',
 'commit',
 'create_aggregate',
 'create_collation',
 'create_function',
 'cursor',
 'enable_load_extension',
 'execute',
 'executemany',
 'executescript',
 'in_transaction',
 'interrupt',
 'isolation_level',
 'iterdump',
 'load_extension',
 'rollback',
 'row_factory',
 'set_authorizer',
 'set_progress_handler',
 'set_trace_callback',
 'text_factory',
 'total_changes']

In [7]:
# cursor object
c = conn.cursor()
type(c)

sqlite3.Cursor

In [8]:
[i for i in dir(c) if i[0] != '_']

['arraysize',
 'close',
 'connection',
 'description',
 'execute',
 'executemany',
 'executescript',
 'fetchall',
 'fetchmany',
 'fetchone',
 'lastrowid',
 'row_factory',
 'rowcount',
 'setinputsizes',
 'setoutputsize']

In [29]:
# sql = """
# DROP TABLE table1
# """
# c.execute(sql)

In [30]:
# execute sql with cursor object
sql = """
CREATE table table1 (
column1 integer,
column2 text
)
"""
c.execute(sql)

sql = """
INSERT INTO table1 (column1, column2)
VALUES 
(1, "row1"),
(2, "row2")
"""
c.execute(sql)

In [34]:
# use c.fetchall() to get every row returned from query
sql = """
SELECT * from table1
"""
c.execute(sql)
c.fetchall()

[(1, 'row1'), (2, 'row2')]

In [39]:
# use c.fetchone() to get rows one at a time (pops row from list)
sql = """
SELECT * from table1
"""
c.execute(sql)
print(c.fetchone())
print(c.fetchone())

(1, 'row1')
(2, 'row2')


In [84]:
# use with pandas
data = [list(range(5)) for _ in range(5)]
df = pd.DataFrame(data)
df.to_sql('table2', conn, if_exists='replace', index=False)

sql = 'SELECT * from table2'
pd.read_sql(sql, conn)

,0,1,2,3,4
0,0,1,2,3,4
1,0,1,2,3,4
2,0,1,2,3,4
3,0,1,2,3,4
4,0,1,2,3,4


# sqlite databases

**Each value stored in an SQLite database (or manipulated by the database engine) has one of the following storage classes:**
- NULL. The value is a NULL value.
- INTEGER. The value is a signed integer, stored in 1, 2, 3, 4, 6, or 8 bytes depending on the magnitude of the value.
- REAL. The value is a floating point value, stored as an 8-byte IEEE floating point numbe
- TEXT. The value is a text string, stored using the database encoding (UTF-8, UTF-16BE or UTF-16LE).
- BLOB. The value is a blob of data, stored exactly as it was input.

SQLite does not have a separate Boolean storage class. Instead, Boolean values are stored as integers 0 (false) and 1 (true).

**SQLite has no DATETIME datatype. Instead, dates and times can be stored in any of these ways:**
- As a TEXT string in the ISO-8601 format. Example: '2018-04-02 12:13:46'.
- As an INTEGER number of seconds since 1970 (also known as "unix time").
- As a REAL value that is the fractional Julian day number.



# Examples

ToDo: 
- pandas df to table with primary key


In [105]:
# get table names of db
sql = '''
SELECT name from sqlite_master 

'''
c.execute(sql)
c.fetchall()

[('table1',), ('table2',), ('df',), ('test_view2',)]

In [95]:
# get column names and types of table
table_name =  'table1'
sql = """
SELECT sql from sqlite_master
WHERE name = '{}' and type = 'table'
""".format(table_name)
c.execute(sql)
print(c.fetchone()[0])

CREATE TABLE table1 (
column1 integer,
column2 text
)


In [91]:
# create table from pandas df with a primary key
import re

def get_create_table_string(tablename, connection):
    sql = """
    select * from sqlite_master where name = "{}" and type = "table"
    """.format(tablename) 
    result = connection.execute(sql)

    create_table_string = result.fetchmany()[0][4]
    return create_table_string

def add_pk_to_create_table_string(create_table_string, colname):
    regex = "(\n.+{}[^,]+)(,)".format(colname)
    return re.sub(regex, "\\1 PRIMARY KEY,",  create_table_string, count=1)

def add_pk_to_sqlite_table(tablename, index_column, connection):
    cts = get_create_table_string(tablename, connection)
    cts = add_pk_to_create_table_string(cts, index_column)
    template = """
    BEGIN TRANSACTION;
        ALTER TABLE {tablename} RENAME TO {tablename}_old_;

        {cts};

        INSERT INTO {tablename} SELECT * FROM {tablename}_old_;
        

        DROP TABLE {tablename}_old_;

    COMMIT TRANSACTION;
    """

    create_and_drop_sql = template.format(tablename = tablename, cts = cts)
    connection.executescript(create_and_drop_sql)

data = [[i+1]*5 for i in range(5)]
df = pd.DataFrame(data, columns = ['col{}'.format(i+1) for i in range(len(data))])
df.to_sql("df", conn, if_exists="replace", index=False)

add_pk_to_sqlite_table("df", "col1", conn)
c.execute("select sql from sqlite_master where name = 'df' and type = 'table'")
print(c.fetchone()[0])

CREATE TABLE "df" (
"col1" INTEGER PRIMARY KEY,
  "col2" INTEGER,
  "col3" INTEGER,
  "col4" INTEGER,
  "col5" INTEGER
)


In [101]:
# append to table using pandas
df.apply(lambda x: x*1000).to_sql("df", conn, if_exists="append", index=False)
pd.read_sql("SELECT * FROM df", conn)

,col1,col2,col3,col4,col5
0,1,1,1,1,1
1,2,2,2,2,2
2,3,3,3,3,3
3,4,4,4,4,4
4,5,5,5,5,5
5,10,10,10,10,10
6,20,20,20,20,20
7,30,30,30,30,30
8,40,40,40,40,40
9,50,50,50,50,50


In [99]:
sql = """
CREATE VIEW test_view2
AS
SELECT
    col1
FROM
    df
"""
c.execute(sql)

In [106]:
sql = 'SELECT * FROM test_view2'
pd.read_sql(sql, conn)

,col1
0,1
1,2
2,3
3,4
4,5
5,10
6,20
7,30
8,40
9,50


In [114]:
# get sql code from tables/views
sql = 'SELECT * FROM sqlite_master'
pd.read_sql(sql, conn)

,type,name,tbl_name,rootpage,sql
0,table,table1,table1,2,"CREATE TABLE table1 (\ncolumn1 integer,\ncolum..."
1,table,table2,table2,3,"CREATE TABLE ""table2"" (\n""0"" INTEGER,\n ""1"" I..."
2,table,df,df,5,"CREATE TABLE ""df"" (\n""col1"" INTEGER PRIMARY KE..."
3,view,test_view2,test_view2,0,CREATE VIEW test_view2\nAS\nSELECT\n col1\n...


In [110]:
sql = 'select * from sqlite_master'
c.execute(sql)
c.fetchall()

[('table',
  'table1',
  'table1',
  2,
  'CREATE TABLE table1 (\ncolumn1 integer,\ncolumn2 text\n)'),
 ('table',
  'table2',
  'table2',
  3,
  'CREATE TABLE "table2" (\n"0" INTEGER,\n  "1" INTEGER,\n  "2" INTEGER,\n  "3" INTEGER,\n  "4" INTEGER\n)'),
 ('table',
  'df',
  'df',
  5,
  'CREATE TABLE "df" (\n"col1" INTEGER PRIMARY KEY,\n  "col2" INTEGER,\n  "col3" INTEGER,\n  "col4" INTEGER,\n  "col5" INTEGER\n)'),
 ('view',
  'test_view2',
  'test_view2',
  0,
  'CREATE VIEW test_view2\nAS\nSELECT\n    col1\nFROM\n    df')]